In [3]:
import json
import requests
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
from openai import OpenAI
GPT_MODEL = "gpt-3.5-turbo"



client = OpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
)



In [5]:
# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):

    # 设定请求的header信息，包括 API_KEY
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    }

    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model": model, "messages": messages}

    # 如果传入了functions，将其加入到json_data中
    if functions is not None:
        json_data.update({"functions": functions})

    # 如果传入了function_call，将其加入到json_data中
    if function_call is not None:
        json_data.update({"function_call": function_call})
        
  

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = requests.post(
            "https://api.openai-proxy.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [6]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:
    
        print('message:',message)    
    
        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))
        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))
            
    

In [7]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]

In [8]:
 #定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions,function_call="auto"
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

pretty_print_conversation(messages)

message: {'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

message: {'role': 'user', 'content': "What's the weather like today"}
user: What's the weather like today

message: {'role': 'assistant', 'content': "Sure, before I can provide you with the current weather, could you please provide me with the location (city and state) you'd like to know the weather for?"}
assistant[content]: Sure, before I can provide you with the current weather, could you please provide me with the location (city and state) you'd like to know the weather for?



In [6]:
type(assistant_message)

dict

In [9]:
# 向messages列表添加一条用户角色的消息，用户告知他们在苏格兰的格拉斯哥
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I'm in Shanghai, China."  # 用户的消息内容
})

# 再次使用定义的chat_completion_request函数发起一个请求，传入更新后的messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions,function_call="auto"
)

# 解析返回的JSON数据，获取助手的新的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的新的回复消息添加到messages列表中
messages.append(assistant_message)

pretty_print_conversation(messages)

message: {'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

message: {'role': 'user', 'content': "What's the weather like today"}
user: What's the weather like today

message: {'role': 'assistant', 'content': "Sure, before I can provide you with the current weather, could you please provide me with the location (city and state) you'd like to know the weather for?"}
assistant[content]: Sure, before I can provide you with the current weather, could you please provide me with the location (city and state) you'd like to know the weather for?

message: {'role': 'user', 'content': "I'm in Shanghai, China."}
user: I'm in Shanghai, China.

message: {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{"location":"Shangha

In [16]:
messages.pop()

{'role': 'user', 'content': "I'm in Shanghai, China."}

In [59]:
# 初始化一个空的messages列表
messages = []

# 向messages列表添加一条系统角色的消息，要求不做关于函数参数值的假设，如果用户的请求模糊，应该寻求澄清
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

# 向messages列表添加一条用户角色的消息，用户询问在未来x天内苏格兰格拉斯哥的天气情况
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "what is the weather going to be like in Shanghai, China over the next x days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure, I can help you with that. How many days do you want the weather forecast for?

Executing function call... [{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': 'what is the weather going to be like in Shanghai, China over the next x days'}, {'role': 'assistant', 'content': 'Sure, I can help you with that. How many days do you want the weather forecast for?'}]


In [123]:
# 向messages列表添加一条用户角色的消息，用户指定接下来的天数为5天
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "5 days"
})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取第一个选项
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 打印助手的回复消息
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in Shanghai, China over the next x days

assistant[content]: Sure, I can help you with that. How many days do you want the weather forecast for?

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":5}'}

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":5}'}

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":5}'}

user: 5 days

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":5}'}

user: 5 days

assistant[function_call]: {'name'

In [17]:
#调用心知天气预报API
def get_current_weatherForAPI(location:str,format:str='celsius') -> str:
    requestsUrl = f"https://api.seniverse.com/v3/weather/now.json?key=SUaEVEcUHIhRwQROw&location={location}&language=zh-Hans&unit=c"
    print('requestsUrl:',requestsUrl)
    weatherResponse = requests.get(requestsUrl)
    return weatherResponse.json()['results'][0]['now']['temperature']

In [21]:
get_current_weatherForAPI('北京')

requestsUrl: https://api.seniverse.com/v3/weather/now.json?key=SUaEVEcUHIhRwQROw&location=北京&language=zh-Hans&unit=c


'14'

In [13]:
#调用心知天气预报API
def get_n_day_weather_forecast(location:str='beijing',start:int=0,num_days:int=5):
     print(f'{location}的天气预报：{num_days}')
     requestsUrl = f"https://api.seniverse.com/v3/weather/daily.json?key=SUaEVEcUHIhRwQROw&location={location}&language=zh-Hans&unit=c&start={start}&days={num_days}"
     print(requestsUrl)
     weatherResponse = requests.get(requestsUrl)    
     return weatherResponse.json()

In [51]:
high_list =[daily['high'] for daily in get_n_day_weather_forecast()['results'][0]['daily']]

high_list

['14', '17', '21']

In [121]:
def execute_function_call(messages):
    for message in messages:
        if 'function_call' in message:
            if message['function_call']:
                function_name = message['function_call']['name']
                function_args = message['function_call']['arguments']
            
                # Call the appropriate function based on the function name
                if function_name == 'get_n_day_weather_forecast':
                    print(f"Executing function: {function_name} with arguments: {function_args}")
                    function_args_object = json.loads(function_args)
                    weather_data = get_n_day_weather_forecast(location=function_args_object['location'].split(',')[0],num_days=function_args_object['num_days'])
                    
                    return weather_data
                elif function_name == 'get_current_weather':
                    print(f"Executing function: {function_name} with arguments: {function_args}")
                    function_args_object = json.loads(function_args)
                    weather_data = get_current_weather(function_args_object['location'])
                    return weather_data

In [122]:
execute_function_call(messages=messages)

Executing function: get_n_day_weather_forecast with arguments: {"location":"Shanghai, China","format":"celsius","num_days":5}
Shanghai的天气预报：5
https://api.seniverse.com/v3/weather/daily.json?key=SUaEVEcUHIhRwQROw&location=Shanghai&language=zh-Hans&unit=c&start=0&days=5


{'results': [{'location': {'id': 'WTW3SJ5ZBJUY',
    'name': '上海',
    'country': 'CN',
    'path': '上海,上海,中国',
    'timezone': 'Asia/Shanghai',
    'timezone_offset': '+08:00'},
   'daily': [{'date': '2024-03-27',
     'text_day': '小雨',
     'code_day': '13',
     'text_night': '小雨',
     'code_night': '13',
     'high': '15',
     'low': '8',
     'rainfall': '12.81',
     'precip': '0.99',
     'wind_direction': '无持续风向',
     'wind_direction_degree': '',
     'wind_speed': '8.4',
     'wind_scale': '2',
     'humidity': '79'},
    {'date': '2024-03-28',
     'text_day': '小雨',
     'code_day': '13',
     'text_night': '晴',
     'code_night': '1',
     'high': '16',
     'low': '11',
     'rainfall': '1.46',
     'precip': '0.50',
     'wind_direction': '无持续风向',
     'wind_direction_degree': '',
     'wind_speed': '8.4',
     'wind_scale': '2',
     'humidity': '93'},
    {'date': '2024-03-29',
     'text_day': '多云',
     'code_day': '4',
     'text_night': '阴',
     'code_night': '9'

In [15]:



# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit",format=""):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    
    response_message = response.choices[0].message
    print(response_message)
    print(type(response_message))
    # Step 2: check if GPT wanted to call a function
    if response_message.function_call:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message.function_call.name
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message.function_call.arguments)
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
            
        )
        
        print('function_response',function_response)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  
        # extend conversation with function response
        print('messages',messages)
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response
    # return response


print(run_conversation())

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)
<class 'openai.types.chat.chat_completion_message.ChatCompletionMessage'>
function_response {"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}
messages [{'role': 'user', 'content': "What's the weather like in Boston?"}, ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}'}]
ChatCompletion(id='chatcmpl-97ciotmyzP5xQsPEeh07iYP8Fsnoh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston is sunny and windy, with a temperature

In [ ]:
def execute_function_callV2(messages):
    for message in messages:
        if 'function_call' in message:
            if message['function_call']:
                
                available_functions = {
                    "get_n_day_weather_forecast": get_n_day_weather_forecast,
                    "get_current_weather": get_current_weatherForAPI,
            }  # only one function in this example, but you can have multiple
                
                function_name = message['function_call']['name']
                function_args = json.loads(message['function_call']['arguments'])
                if function_args.get('location'):
                    function_args['location'] = function_args['location'].split(',')[0]
                    
                fuction_to_call = available_functions[function_name]
                
                print(f"Calling function {function_name} with arguments {function_args}")

                function_response = fuction_to_call(    
                    **function_args
                )    
                print(function_response)    
            
            

In [25]:
execute_function_callV2(messages=messages)

Calling function get_current_weather with arguments {'location': 'Shanghai', 'format': 'celsius'}
<class 'dict'>
requestsUrl: https://api.seniverse.com/v3/weather/now.json?key=SUaEVEcUHIhRwQROw&location=Shanghai&language=zh-Hans&unit=c
13
